#Write data from storage to eventhubs to simulate streaming approach

In [2]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [3]:
raw_folder = '/mnt/test/*.csv' #'/mnt/raw/reddit'

# Event Hubs Connection Configuration
ehConf = {
  'eventhubs.connectionString': dbutils.secrets.get(scope="mle2ebigdatakv", key="redditstreamingkey")
}


Read from storage using Spark, define 'body' and write to eventhubs:

In [5]:
redditSchema = StructType([
  StructField("file_index", StringType(), True),
  StructField("text", StringType(), True),
  StructField("id", StringType(), True),
  StructField("subreddit", StringType(), True),
  StructField("meta", StringType(), True),
  StructField("time", StringType(), True),
  StructField("author", StringType(), True),
  StructField("ups", StringType(), True),
  StructField("downs", StringType(), True),
  StructField("authorlinkkarma", StringType(), True),
  StructField("authorkarma", StringType(), True),
  StructField("authorisgold", StringType(), True)
])

In [6]:
reddit = spark.read.csv(raw_folder,redditSchema)
reddit.limit(5).toPandas()
reddit.show()

+----------+--------------------+-------+---------+-------------+------------+------------------+----+-----+---------------+-----------+------------+
file_index| text| id|subreddit| meta| time| author| ups|downs|authorlinkkarma|authorkarma|authorisgold|
+----------+--------------------+-------+---------+-------------+------------+------------------+----+-----+---------------+-----------+------------+
 1|because she s the...|d02u69l| anime|entertainment|1455683054.0| Redire77| 7.0| 0.0| 352| 14017| 0.0|
 2|i am referring to...| 466ijy| anime|entertainment|1455682823.0| shiba_arata| 0.0| 0.0| 1| 20| 0.0|
 3|cheating but zold...|d02g879| anime|entertainment|1455661236.0| ShaKing807| 6.0| 0.0| 1308| 62021| 1.0|
 4|kurosaki ichigo ...|d02v88z| anime|entertainment|1455684994.0| Tf2idlingftw| 2.0| 0.0| 4156| 1021| 0.0|
 5|there are a shit ...|d02ggn7| anime|entertainment|1455661564.0| Angst-Incarnate| 1.0| 0.0| 137| 41859| 0.0|
 6|they went full fr...|d0280d1| anime|entertainment|1455650264.0|ASK-ME-IF-I-DID-IT| 7.0| 0.0| 1154| 8941| 0.0|
 7|by far the best e...|d02307g| anime|entertainment|1455643470.0| Smudy| 2.0| 0.0| 740| 4325| 0.0|
 8|gal stan might as...|d026ijg| anime|entertainment|1455648246.0| Russiandragon55| 3.0| 0.0| 1000| 605| 0.0|
 9|hi th3jester_ it...|d020243| anime|entertainment|1455639272.0| AutoModerator|19.0| 0.0| 1476| 278459| 1.0|
 10|related shoune...|d029srr| anime|entertainment|1455652653.0| _F1_| 2.0| 0.0| 2543| 65930| 0.0|
 11|uchuu senkan yama...|d02f5bq| anime|entertainment|1455659732.0| kikoano| 2.0| 0.0| 2779| 1513| 0.0|
 12|gt the series wa...|d01wy1q| anime|entertainment|1455634221.0| Pyrelord|11.0| 0.0| 24966| 114769| 0.0|
 13|stop abandoningtk...|d024xdx| anime|entertainment|1455646097.0| NolantheBoar| 1.0| 0.0| 3495| 9799| 0.0|
 14|time of eve htt...|d025a1a| anime|entertainment|1455646574.0| mmreviews| 2.0| 0.0| 44| 4639| 0.0|
 15|fatezero also gu...|d021xvw| anime|entertainment|1455641969.0| EasymodeX| 8.0| 0.0| 1| 15123| 0.0|
 16|puella magi madok...|d022206| anime|entertainment|1455642135.0| TaigaBlitz| 7.0| 0.0| 1| 4282| 0.0|
 17|try giving tokyo...|d0224sb| anime|entertainment|1455642246.0| omaketheater| 1.0| 0.0| 559| 1103| 0.0|
 18|darker than black...|d024bpt| anime|entertainment|1455645274.0| corvaun| 2.0| 0.0| 1| 19| 0.0|
 19| 12 kingdoms|d02c6a7| anime|entertainment|1455655777.0| Kyrdra| 1.0| 0.0| 654| 6802| 0.0|
 20|i think you d enj...|d020gkx| anime|entertainment|1455639864.0| Ralon17|24.0| 0.0| 1072| 7202| 0.0|
+----------+--------------------+-------+---------+-------------+------------+------------------+----+-----+---------------+-----------+------------+
only showing top 20 rows

In [7]:
tmp = reddit.where(reddit.id!='1')
tmp = tmp.drop('file_index')
reddit = tmp

In [8]:
colList = [col(column) for column in reddit.columns ]

In [9]:
#TODO merge string columns to formulate a body column that we can later parse according to schema
from pyspark.sql.functions import concat_ws, col, lit # alias('body')
bodydf = reddit.select(concat_ws(',', *colList).alias('body'))

In [10]:
filtereddf = bodydf.where(bodydf.body!='0,1,2,3,4,5,6,7,8,9,10')

In [11]:
# from pyspark.sql.functions import not as n

filter_adult_content = filtereddf.filter("LOWER(body) NOT LIKE '%sex%' ")
filter_adult_content.show()

+--------------------+
 body|
+--------------------+
because she s the...|
i am referring to...|
cheating but zold...|
kurosaki ichigo ...|
there are a shit ...|
they went full fr...|
by far the best e...|
gal stan might as...|
hi th3jester_ it...|
related shoune...|
uchuu senkan yama...|
gt the series wa...|
stop abandoningtk...|
time of eve htt...|
fatezero also gu...|
puella magi madok...|
try giving tokyo...|
darker than black...|
12 kingdoms,d02c6...|
i think you d enj...|
+--------------------+
only showing top 20 rows

In [12]:
filter_adult_content = filter_adult_content.withColumn("body",regexp_replace("body","fuck","life"))
filter_adult_content = filter_adult_content.withColumn("body",regexp_replace("body","idiot","life"))
filter_adult_content.show()

filtereddf = filter_adult_content

+--------------------+
 body|
+--------------------+
because she s the...|
i am referring to...|
cheating but zold...|
kurosaki ichigo ...|
there are a shit ...|
they went full fr...|
by far the best e...|
gal stan might as...|
hi th3jester_ it...|
related shoune...|
uchuu senkan yama...|
gt the series wa...|
stop abandoningtk...|
time of eve htt...|
fatezero also gu...|
puella magi madok...|
try giving tokyo...|
darker than black...|
12 kingdoms,d02c6...|
i think you d enj...|
+--------------------+
only showing top 20 rows

In [13]:
# add a forever loop / for each row write to eventhubs:
df = filtereddf.select("body").dropna()
df.count()


Out[11]: 2411685

In [14]:
def sampleData(df, index):
    count = df.count();
    howManyTake = 1
    if(count > index):
      howManyTake = index
    else:
      howManyTake = count
    
    return df.sample(False, 1.0*howManyTake/count,42).limit(howManyTake)

In [15]:
import time

for lp in range(100, 2000):
  print("Batch number: {}".format(lp))
  sample = sampleData(df, lp)
  sample.write.format("eventhubs").options(**ehConf).save()
  time.sleep(3)

Batch number: 100
Batch number: 101
Batch number: 102
Batch number: 103
Batch number: 104
Batch number: 105
Batch number: 106
Batch number: 107
Batch number: 108
Batch number: 109
Batch number: 110
Batch number: 111
Batch number: 112
Batch number: 113
Batch number: 114
Batch number: 115
Batch number: 116
Batch number: 117
Batch number: 118
Batch number: 119
Batch number: 120
Batch number: 121
Batch number: 122
Batch number: 123
Batch number: 124
Batch number: 125
Batch number: 126
Batch number: 127
Batch number: 128
Batch number: 129
Batch number: 130
Batch number: 131
Batch number: 132
Batch number: 133
Batch number: 134
Batch number: 135
Batch number: 136
Batch number: 137
Batch number: 138
Batch number: 139
Batch number: 140
Batch number: 141
Batch number: 142
Batch number: 143
Batch number: 144
Batch number: 145
Batch number: 146
Batch number: 147
Batch number: 148
Batch number: 149
Batch number: 150
Batch number: 151
Batch number: 152
Batch number: 153
Batch number: 154
Batch number: 155
Batch number: 156
Batch number: 157
Batch number: 158
Batch number: 159
Batch number: 160
Batch number: 161
Batch number: 162
Batch number: 163
Batch number: 164
Batch number: 165
Batch number: 166
Batch number: 167
Batch number: 168
Batch number: 169
Batch number: 170
Batch number: 171
Batch number: 172
Batch number: 173
Batch number: 174
Batch number: 175
Batch number: 176
Batch number: 177
Batch number: 178
Batch number: 179
Batch number: 180
Batch number: 181
Batch number: 182
Batch number: 183
Batch number: 184
Batch number: 185
Batch number: 186
Batch number: 187
Batch number: 188
Batch number: 189
Batch number: 190
Batch number: 191
Batch number: 192
Batch number: 193
Batch number: 194
Batch number: 195
Batch number: 196
Batch number: 197
Batch number: 198
Batch number: 199
Batch number: 200
Batch number: 201
Batch number: 202
Batch number: 203
Batch number: 204
Batch number: 205
Batch number: 206
Batch number: 207
Batch number: 208
Batch number: 209
Batch number: 210
Batch number: 211
Batch number: 212
Batch number: 213
Batch number: 214
Batch number: 215
Batch number: 216
Batch number: 217
Batch number: 218
Batch number: 219
Batch number: 220
Batch number: 221
Batch number: 222
Batch number: 223
Batch number: 224
Batch number: 225
Batch number: 226
Batch number: 227
Batch number: 228
Batch number: 229
Batch number: 230
Batch number: 231
Batch number: 232
Batch number: 233
Batch number: 234
Batch number: 235
Batch number: 236
Batch number: 237
Batch number: 238
Batch number: 239
Batch number: 240
Batch number: 241
Batch number: 242
Batch number: 243
Batch number: 244
Batch number: 245
Batch number: 246
Batch number: 247
Batch number: 248
Batch number: 249
Batch number: 250
Batch number: 251
Batch number: 252
Batch number: 253
Batch number: 254
Batch number: 255
Batch number: 256
Batch number: 257
Batch number: 258
Batch number: 259
Batch number: 260
Batch number: 261
Batch number: 262
Batch number: 263
Batch number: 264
Batch number: 265
Batch number: 266
Batch number: 267
Batch number: 268
Batch number: 269
Batch number: 270
Batch number: 271
Batch number: 272
Batch number: 273
Batch number: 274
Batch number: 275
Batch number: 276
Batch number: 277
Batch number: 278
Batch number: 279
Batch number: 280
Batch number: 281
Batch number: 282
Batch number: 283
Batch number: 284
Batch number: 285
Batch number: 286
Batch number: 287
Batch number: 288
Batch number: 289
Batch number: 290
Batch number: 291
Batch number: 292
Batch number: 293
Batch number: 294
Batch number: 295
Batch number: 296
Batch number: 297
Batch number: 298
Batch number: 299
Batch number: 300
Batch number: 301
Batch number: 302
Batch number: 303
Batch number: 304
Batch number: 305
Batch number: 306
Batch number: 307
Batch number: 308
Batch number: 309
Batch number: 310
Batch number: 311
Batch number: 312
Batch number: 313
Batch number: 314
Batch number: 315
Batch number: 316
Batch number: 317
Batch number: 318
Batch number: 319
Batch number: 320
Batch number: 321
Batc